COMP 215 - LAB 1
----------------
#### Name:
#### Date:

This lab exercise is mostly to introduce some of the power in Jupyter Notebooks.
Note that a Notebook is composed of "cells" - some are "text", like this one, while others are "code"

##### Lab Objectives

* create a 2D time-series plot of earthquake data
* review common data types (`int` and `str`) and data structures (`list` and `dict`)
* introduce a few new python programming techniques

**New Python Concepts**:
* *list comprehension* provides a compact way to represent map and filter algorithms
* `zip` is a built-in algorithm for combining "parallel" linear data structures
* `datetime.date` objects represent a calendar date (these are very powerful)

As will be usual, the fist code cell, below, simply imports all the modules we'll be using...

In [2]:
import json, requests
import matplotlib.pyplot as plt
import  matplotlib.dates as mdates
from datetime import datetime
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

2### Making an API Query
We'll start by fetching and inspecting some earthquake data from the USGS API

Goals:
- Make an API request
- Convert the JSON response into a Python dictionary
- Inspect the structure of the returned data

USGS API Documentation:  https://earthquake.usgs.gov/fdsnws/event/1/#parameters

Our first query:
  - `format=geojson`       # the format we want the data in
  - `starttime=2022-01-01` # the start date (date format: yyyy-mm-dd)
  - `minmagnitude=5.0`   # smallest magnitude earthquake we're interested in

In [39]:
USGS_API_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

params = dict(
    format="geojson",
    starttime="1970-01-01",
    minmagnitude=7.0,
)

# make a request (like browsing to a web page) and print the results
response = requests.get(USGS_API_URL, params=params)

print('Response data type:', type(response.text))
response.text[:1000]

Response data type: <class 'str'>


'{"type":"FeatureCollection","metadata":{"generated":1768183988000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1970-01-01&minmagnitude=7.0","title":"USGS Earthquakes","status":200,"api":"1.14.1","count":773},"features":[{"type":"Feature","properties":{"mag":7.6,"place":"2025 Aomori Prefecture, Japan Earthquake","time":1765203310397,"updated":1767717524892,"tz":null,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/us6000rtdt","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000rtdt&format=geojson","felt":255,"cdi":7.4,"mmi":6.911,"alert":"yellow","status":"reviewed","tsunami":1,"sig":1077,"net":"us","code":"6000rtdt","ids":",at00t6yfla,pt25342050,us6000rtdt,usauto6000rtdt,","sources":",at,pt,us,usauto,","types":",dyfi,earthquake-name,finite-fault,ground-failure,impact-link,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,origin,phase-data,shakemap,","nst":171,"dmin":0.871,"rms":0.72,"gap":17,"mag

Notice that the response looks like a dictionary, but is actually just a string of text (most data on the web is exchanged as plain text!).  This particular data format is called "[JSON](https://en.wikipedia.org/wiki/JSON)"

The `json.loads` function "parses" such text and loads the data into a dictionary...

In [4]:
data = json.loads(response.text)
print(f"Query returned a dict with {len(data)} keys")
print(data.keys())   # pretty-print the keys from dict we got back...

Query returned a dict with 4 keys
dict_keys(['type', 'metadata', 'features', 'bbox'])


In [5]:
# the list of earthquake data are under the "features" key...
earthquake_list = data["features"]
print(f"Query returned {len(earthquake_list)} earthquake records.")
# and here is what the start of that list of earthquake records looks like...
pprint(earthquake_list[:2])  # do you recall the "slice" operation?  If not, look it up in the ThinkCsPy textbook!

Query returned 7204 earthquake records.
[{'geometry': {'coordinates': [156.1179, 49.2901, 56.243], 'type': 'Point'},
  'id': 'us7000rp2k',
  'properties': {'alert': None,
                 'cdi': None,
                 'code': '7000rp2k',
                 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000rp2k&format=geojson',
                 'dmin': 3.95,
                 'felt': None,
                 'gap': 120,
                 'ids': ',usauto7000rp2k,us7000rp2k,',
                 'mag': 5.1,
                 'magType': 'mb',
                 'mmi': None,
                 'net': 'us',
                 'nst': 75,
                 'place': '154 km S of Severo-Kuril’sk, Russia',
                 'rms': 0.71,
                 'sig': 400,
                 'sources': ',usauto,us,',
                 'status': 'reviewed',
                 'time': 1768160276651,
                 'title': 'M 5.1 - 154 km S of Severo-Kuril’sk, Russia',
                 'tsunami': 0,
  

#### Structured Data
Notice how the data is organized

- a dictionary with list of data "records"
- each data "record" describes one earthquake
- each "earthquake" is itself a dictionary of more dictionaries and lists

This is typical for API results: a `dict` of `list`s of `dict`s of `list`s of `dict`s

Understanding how data is structured is the first lesson for COMP215 - here we see how complex that can get even for a relatively simple data set.

### Extract earthquake data items from a list of dictionaries

Goals:

- create a data set suitable for counting the number of earthquakes per year
- introduction to "list comprehension"
- introduction to `datetime` objects

We will use "list comprehension" to extract the list of dates and associated quake magnitude into "parallel lists"..

In [24]:
quake_timestamps = [quake['properties']['time'] for quake in earthquake_list] # List Comprehension #1: extract timestamps
quake_mags = [quake['properties']['mag'] for quake in earthquake_list] # List Comprehension #2:  extract the magnitudes

print('Timestamps:', quake_timestamps[:5])
print('Magnitudes:', quake_mags[:5])

Timestamps: [1768160276651, 1768104937170, 1768077817306, 1768059248699, 1768058113989]
Magnitudes: [5.1, 5.2, 5, 5, 5.2]


### Datetime.date
Working with date strings is a pain.  So many formats!  Even within Canada, you might see:
"Jan. 9, 2023" or "09-01-2023" or "2023-01-09" or ....
Imagine trying to do a calculation like "how many days between these 2 dates"!!

The build-in `datetime` package makes working with dates much easier.
  * construct a `datetime` object from a "timestamp"  (typically the number of seconds since "epoch")
  * "parse" the date string data (`strptime` ==  "string-parse-datetime object")
  * get parts of the date (i.e., just the date without the time, or even just the year or month)

In [25]:
# construct a datetime object from the "timestamp" - convert from ms to seconds, then convert from timestamp to datetime
quake_dates = [
    datetime.fromtimestamp(timestamp/1000) for timestamp in quake_timestamps
]  # List Comprehension #3: transform each timestamp into a datetime object

# but we only want the "year" part...
quake_years = [dt.year for dt in quake_dates]  # List Comprehension #4: extrac year from each date
quake_years[:5]

[2026, 2026, 2026, 2026, 2026]

In [26]:
# Now "zip" the year and magnitude data back together - this is a "parallel list" operation
#     "zip" takes 2 or more lists and "zips" them together like a zipper - look it up!
print('Zipped:')
pprint(list(zip(quake_years[:5], quake_mags[:5])))

Zipped:
[(2026, 5.1), (2026, 5.2), (2026, 5), (2026, 5), (2026, 5.2)]


## Exercise 1 : loops vs. comprehensions

In the code cell below, re-write the "List Comprehensions" above using one or more`for` loops so you understand how they work.

Goal:

- replicate the final zipped list of 2-tuples (year, magnitude) using the "list accumulator" algorithm you learned in 115

Compare to see how a "list comprehension" is a compact way to write a "list accumulator" algorithm (and more efficient too!)

In [ ]:
# Ex. 1 your code here

from itertools import islice

my_year_mag = []

for my_quake_year in quake_dates:
  for my_earthquake_mag in earthquake_list:
    my_year_mag.append((my_quake_year.year, my_earthquake_mag['properties']['mag']))

print(my_year_mag)
#print(list(islice(my_year_mag, 5)))

## Exercise 2:  counting dates

In the code cell below, write an algorithm that counts the number of quakes in each year.

Goal:

- produce a dictionary that maps each year in the data set to a count of the number of quakes in that year
- an mock example of the data structure you are aiming for is given below

_Hints_:

- this is also an "accumulator" algorithm, where you are accumulating a count for each year
- a `defaultdict` can simplify the algorithm - look it up if you are interested (not required)


In [34]:
# Ex. 2 your code here

from collections import defaultdict

earthquake_counts = defaultdict(int)

for my_quake_year in quake_years:
  earthquake_counts[my_quake_year] += 1

print(earthquake_counts)

# Replace this "mock" data with your results - these data are plotted in next step...
earthquake_counts = {
    2022: 1726,
    2023: 1780,
    2024: 1507,
    2025: 2133,
}

defaultdict(<class 'int'>, {2026: 58, 2025: 2133, 2024: 1507, 2023: 1780, 2022: 1726})


### Generating a plot

Finally, we'll plot the (year, earthquake_count) data as a nice x-y line graph....

In [ ]:
plt.figure()
plt.plot(earthquake_counts.keys(), earthquake_counts.values())
plt.xlabel("Year")
plt.ylabel(f"Number of Earthquakes (Magnitude ≥ {params['minmagnitude']})")
plt.title(f"Earthquakes per Year Since {params['starttime']}")
plt.show()

## Exercise 3: re-run the analysis for a different data set

Repeat the analysis above, but this time for large-magnitude earthquakes (e.g., >=7) over a longer timespan (e.g., since 1970).
(Note that the API limits results to 20,000 records, if it fails, reduce the size of your query.)

* Just re-run the analysis with new parameters!
* Submit your lab workbook run with your final analysis parameters


## Challenge Exercises (optional) - Take your skills to the next level...

## Exercise 4: Earthquakes by month

1. re-do the data transformations to extract (dt.year, dt.month) 2-tuples from each earthquake timestamp
2. re-do the count analysis to tally the number of quakes in each month
3. create a new plot with the number of quakes per month

- Develop all the code for this exercise in one or more code blocks below.
- You may copy any of the code from blocks above, then revise it to do the monthly analysis.
- Change the variable names you use (e.g., `quake_months` instead of `quake_years`) to avoid getting confused

#### Exercise 5: Smoothing function super-challenge!

Notice that the data plot looks quite erratic.  Earthquakes are "self-organizing", "critical" systems, which we'll learn
more about later in the term.  Their erratic behaviour makes it hard to spot long-term trends.

 * One technique to help highlight trends is to "smooth" the data using a "6-month rolling average".
Each month, we take the average of the previous 6 months of earthquake counts.
 * Add new code cell below, compute the 6-month rolling average for each month from the raw monthly count data.
 * Create a plot to display the rolling average data and compare your plot with the one produced in Ex. 4.

 Hints: you are free to do this however you like, but a quite elegant solution uses list comprehension, range, and slices

In [ ]:
# Ex. 4 (challenge) your code here